### Bibliotecas utilizadas em todas as celulas

In [1]:
from bot.historico_precos import get_price_history
from bot.models.coin_pair import CoinPair
from db.duckdb_csv import save_to_csv_duckdb
from segredos import BASE_DIR

### Definição da única moeda utilizada

In [2]:
coinpair = CoinPair(
    base='BTC',
    quote='BRL',
)

### Código para criar dataset com entradas volume, preço de abertura, preço de fechamento, preço máximo e preço mínimo e saida com o proximo preço de fechamento ou seja deslocado em uma linha

In [ ]:
df = get_price_history(coin_pair=coinpair)
# df = calculate_indicators(df)
# df = generate_signals(df)
df['y'] = df['close'].shift(-1)
df = df.dropna()
save_to_csv_duckdb(
    df,
    BASE_DIR
    + '/datasets'
    + '/saida_como_preco_do_ativo_'
    + f'{coinpair.bitpreco_websocket}_{coinpair.exchange.value}'
    + '.csv',
)

True